In [14]:
from sklearn.externals import joblib
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings("ignore", category=Warning) 

In [15]:
data = pd.read_csv('book32-listing.csv',encoding = "ISO-8859-1")

columns = ['Id', 'Image', 'Image_link', 'Title', 'Author', 'Class', 'Genre']
data.columns = columns

books = pd.DataFrame(data['Title'])
author = pd.DataFrame(data['Author'])
genre = pd.DataFrame(data['Genre'])

data['Author'] = data['Author'].fillna('No Book')
data['Title'] = data['Title'].fillna('No Book')

feat = ['Genre']
for x in feat:
    le = LabelEncoder()
    le.fit(list(genre[x].values))
    genre[x] = le.transform(list(genre[x]))
    
data['everything'] = pd.DataFrame(data['Title'] + ' ' + data['Author'])


In [16]:
class_to_arr_dict = {
'Arts & Photography':['art','photography'],
'Biographies & Memoirs':['historical'],
'Business & Money':['business','finance','money'],
'Calendars':['general'],
"Children's Books":['general'],
'Comics & Graphic Novels':['general'],
'Computers & Technology':['computing','technology'],
'Cookbooks, Food & Wine':['cooking','food','wine'],
'Crafts, Hobbies & Home':['crafts','hobbies','home'],
'Christian Books & Bibles':['christian','bible'],
'Engineering & Transportation':['engineering','transportation'],
'Health, Fitness & Dieting':['health','fitness','diet'],
'History':['history'],
'Humor & Entertainment':['humor','entertainment'],
'Law':['law'],
'Literature & Fiction':['literature','fiction'],
'Medical Books':['medical','medicine'],
'Mystery, Thriller & Suspense':['general'],
'Parenting & Relationships':['parenting','relationships'],
'Politics & Social Sciences':['political','social'],
'Reference':['general'],
'Religion & Spirituality':['religion','spiritual'],
'Romance':['general'],
'Science & Math':['science','mathematics'],
'Science Fiction & Fantasy':['general'],
'Self-Help':['self-help','aspirational'],
'ports & Outdoors':['outdoors'],
'Teen & Young Adult':['teen', 'youth'],
'Test Preparation':['test','exam','preparation'],
'Travel':['travel'],
'Gay & Lesbian':['gay','lesbian'],
'Education & Teaching':['education','teaching','academic']
}

In [17]:
vectorizer = TfidfVectorizer(min_df=2, max_features=70000, strip_accents='unicode',lowercase =True,
                            analyzer='word', token_pattern=r'\w+', use_idf=True, 
                            smooth_idf=True, sublinear_tf=True, stop_words = 'english')
vectors = vectorizer.fit_transform(data['everything'])   

In [18]:
def inference(term):
    text = []
    text.append(term)
    text[0] = text[0].lower()
    arr = (vectorizer.transform(text))
    clf = joblib.load('best.pkl')
    prediction = clf.predict(arr)
    prediction = le.inverse_transform(prediction)[0]
    return class_to_arr_dict[prediction]